# Assignment 09

Learning goal: develop confidence working with Google Earth Engine and build comfort working with new datasets.

The tools needed to complete this assignment are largely found within the Intro Python Tutorial that you have gone through.
If you haven't already, please pause, and run through this tutorial, copying each cell block of text from the tutorial
into a new notebook.  Here is the web address to the tutorial: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard

Skim this website to learn the fundamental building blocks (i.e., objects and methods) of EE: 
https://developers.google.com/earth-engine/guides/objects_methods_overview

With the above knowledge, I hope that you will be able to use the [tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard) 
as something of a guide, and copy, paste, and modify the steps used in the tutorial to meet the specific goals
of this assignment.

As always, post your questions on slack!


In [1]:
import ee
from IPython.display import Image
import pandas as pd

import folium # For super-cool interactive maps


In [2]:
ee.Initialize() # necessary to let Google know who you are, and that you'll be using GEE tools.


## Exploring snow cover in and around Moscow
There is a daily estimate of snow cover from the MODIS Terra satellite,
using a band ratio metric akin to the NDVI we've used previously.  This metric
is known as the Normalized Difference Snow Index (NDSI).  We'll use this NASA
data product, available through Google Earth Engine.

Find the MOD10A1.006 Terra Snow Cover Daily Global 500m within the Earth Engine Data Catalog:
https://developers.google.com/earth-engine/datasets

Review the Bands that are reported for this dataset.  On the Bands page, each Band of the
dataset is named with bold font, and then, to the right of the name, is a range of various
different kinds of information defining what is included in each band.



In [35]:
# Create an ee.ImageCollection variable named "snow"
#   complete the following line of code
#'MOD10A1.006 Terra Snow Cover Daily Global 500m'
snow =  ee.ImageCollection('MODIS/006/MOD10A1') #500m

In [37]:
global_mean_snowcover = snow.mean().select('NDSI_Snow_Cover')

In [38]:
global_mean_snowcover.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDSI_Snow_Cover',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [40]:
from IPython.display import Image

# Create a URL to the styled image for a region around France.
url = global_mean_snowcover.getThumbUrl({
    'min': 0, 'max': 100, 'dimensions': 512,# 'region': roi,
    'palette': ['black', 'white']})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/dc4300cb88eaa83f6e2053f81f1960f3-630c5af87b25f56b00083d7533d30ab4:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [10]:
snow.first().bandNames().getInfo()

['NDSI_Snow_Cover',
 'NDSI_Snow_Cover_Basic_QA',
 'NDSI_Snow_Cover_Algorithm_Flags_QA',
 'NDSI',
 'Snow_Albedo_Daily_Tile',
 'orbit_pnt',
 'granule_pnt',
 'NDSI_Snow_Cover_Class',
 'Snow_Albedo_Daily_Tile_Class']

In [12]:
snow = snow.select('NDSI_Snow_Cover')
type(snow)

ee.imagecollection.ImageCollection

In [13]:
snow.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDSI_Snow_Cover',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [86400, 43200],
   'crs': 'SR-ORG:6974',
   'crs_transform': [463.312716528,
    0,
    -20015109.354,
    0,
    -463.312716527,
    10007554.677]}],
 'version': 1541936909829165,
 'id': 'MODIS/006/MOD10A1/2000_02_24',
 'properties': {'system:time_start': 951350400000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 951436800000,
  'system:asset_size': 396169135,
  'system:index': '2000_02_24'}}

In [14]:
# Find how many images are in the ImageCollection
snow.size()

In [24]:
test =  ee.Image('MODIS/006/MOD10A1') #500m

In [29]:
#test.getInfo()

In [16]:
365*20

7300

In [15]:
snow.size().getInfo()

7859

In [9]:
# Use getInfo() to report the contents of the first Image of the 'snow' ImageCollection


In [17]:
snow.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDSI_Snow_Cover',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [86400, 43200],
   'crs': 'SR-ORG:6974',
   'crs_transform': [463.312716528,
    0,
    -20015109.354,
    0,
    -463.312716527,
    10007554.677]}],
 'version': 1541936909829165,
 'id': 'MODIS/006/MOD10A1/2000_02_24',
 'properties': {'system:time_start': 951350400000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 951436800000,
  'system:asset_size': 396169135,
  'system:index': '2000_02_24'}}

Recognize above that, after `type` (`Image`), the different bands are all identified according to their
`id` (name) and other information about the contents of the band.

We want a simple, straightforward way to assess the percentage of a pixel (500 m x 500 m) that
is snow covered.  So we need to find which band contains this information.  The Google Earth Engine
Data Catalog suggests which band this might be, but it can be a little opaque.  To clarify the contents
of the bands, go to the
National Snow and Ice Data Center (NSIDC- which is a NASA data repository) and is the actual home/source
of this data product.  Earth Engine serves it, but the product is really from NASA and is distributed
by the NSIDC.  There are two ways this can be accessed from Earth Engine.  I've circled and
drawn an arrow next to these options.

<!-- ![title](find_documentation.png) -->
<img src="find_documentation.png" alt="alt text" width="500"/>

Within the NSIDC dataset website, you'll find more information about the origin of the dataset, and
how to use it.  You'll see multiple tabs, starting with "Overview" (basic metadata), "Download
Data" (don't do it!  We're using Earth Engine!!!), "Citing ...", "User Guide," etc.

The User Guide describes the origin of these data, how they were processed, etc.  Give the User Guide
pdf a look on the NSIDC website to decide which band to use going forward, as your best estimate
of snow cover.


In [18]:
elv = ee.Image('USGS/SRTMGL1_003')

### Mean snow cover in Moscow
Find the mean snow cover over all times at location just outside of Moscow.
To avoid any impacts associated with the town itself (i.e., buildings, parking lots, plowing, etc.),
choose a coordinate to `sample` that is just outside the developed regions of the town.  You might
use Google Earth, or some other GIS software, to pick this coordinate.

In [19]:
# Define the location of interest for Moscow.
#moscow_lon = -117.03# -117.00
#moscow_lat = 46.72#46.732
moscow_lon = -116.98# -117.00
moscow_lat = 46.75#46.732
mm_lat = 46.81
mm_long = -116.86

moscow_poi = ee.Geometry.Point(moscow_lon, moscow_lat)
mm_poi = ee.Geometry.Point(mm_long, mm_lat)

# Define the operative scale of the dataset.  This is important for defining how EE does its analyses.
#    This should be the resolution of the data product.
scale = 500  # scale in meters


In [32]:
type(snow.mean())

ee.image.Image

In [31]:
snow.mean().sample(mm_poi, scale).getInfo()

{'type': 'FeatureCollection',
 'columns': {'NDSI_Snow_Cover': 'Float<0.0, 255.0>'},
 'properties': {'band_order': ['NDSI_Snow_Cover']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0',
   'properties': {'NDSI_Snow_Cover': 6.873100303951368}}]}

In [30]:
snow.mean().sample(mm_poi, scale).first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'NDSI_Snow_Cover': 6.873100303951368}}

In [13]:
moscow_mountain_point = snow.mean().sample(mm_poi, scale).first().get('NDSI_Snow_Cover').getInfo()
print('Average snow mean near Moscow Mountain:', round(moscow_mountain_point))

Average snow mean near Moscow Mountain: 7


In [14]:
#ui.Char.array.values(snow,5,xlabels)

### Visualize the mean snow cover

Calculate the mean snow cover of the entire dataset.  For the region around Moscow, plot this mean snow cover.

In [15]:
total_mean = snow.mean().get('NDSI_Snow_Cover').getInfo()
print('The total mean of snow is:', total_mean)

The total mean of snow is: None


In [16]:
snow_img = snow.select('NDSI_Snow_Cover').first()



In [17]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[46.8, -117], zoom_start=10)    
    
snow_vis_params = {
    'min': 0,'max': 250,
    'palette': ['0000FF', '1F1FFF', '4949FF', 'A3A3FF','BFBFFF']}

#mm_vis_params = {
#    'min': 0,'max': 250,
#    'palette': ['0000FF', '1F1FFF', '4949FF', 'A3A3FF','BFBFFF']}


# Create a map.
lat, lon = 46.81, -116.86
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(snow_img, snow_vis_params, 'Mean Snow Cover')

#my_map.add_ee_layer(mm_poi, mm_vis_params, 'Moscow')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

#### Set up a Folium map to explore the output data with

In [18]:
elev = ee.Image('USGS/SRTMGL1_003') # Read in the STRM dataset as an EE Image

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[46.8, -117], zoom_start=10)

# Set visualization parameters for the elevation dataset
elev_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

snow_vis_params = {
    'min': 0,'max': 250,
    'palette': ['0000FF', '1F1FFF', '4949FF', 'A3A3FF','BFBFFF']}



# Add the elevation model to the map object.
my_map.add_ee_layer(elev, elev_params, 'DEM')
my_map.add_ee_layer(snow_img, snow_vis_params, 'Mean Snow Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
#folium.LayerControl(collapsed = False).add_to(my_map)


# Display the map.
display(my_map)

### Interpretting the snow cover
Add a layer to the Folium map above that includes the mean of all the snow cover measurements.
Note in the map above there there are only a few lines used to define the `elev_params` and then one 
line that uses `add_ee_layer` to add the `elev` dataset to the map.  You can reproduce (and modify)
these lines to show your map of mean snow cover.

Write a few sentences to interpret the spatial patterns in mean snow cover in the several 10s of km
basin surrounding Moscow.

In [19]:

# Add the elevation model to the map object.
my_map.add_ee_layer(snow, snow_params, 'DEM')


NameError: name 'snow_params' is not defined

In [ ]:
snow_params = {
  'min': 0,
  'max': 255,
  'palette': ['33D1FF', '3396FF', '', '334FFF', '4C33FF']}

ee_tiles = [elv, snow]
ee_params = [elev_params, snow_params]
ee_tiles_names = ['Elevation', 'NDSI',]

# Add layers to the map using a loop.
for tile, _param, name in zip(ee_tiles, ee_params, ee_tiles_names):
    my_map.add_ee_layer(tile, _param, name)

## Examining time series of snow cover around Moscow
Your goal now is to produce a time series of snow cover (from MODIS NDSI) near Moscow (but again, just outside Moscow, to
avoide city effects).

Again, following the example from the [Python API tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard).
Produce an array (technically a list of lists in this case) with all of the snow cover measurements of your Moscow
coordinates.

In [ ]:
#test = mm_poi.getRegion(mm_poi, scale).getInfo()

In [20]:
# Function to transform the EE array (list of lists) into a pandas dataframe
def ee_array_to_df(mm_poi, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(mm_poi)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

Use the function above to create a DataFrame that includes snow cover as a function of time.  Show
that DataFrame below.

In [80]:
#df

Create a plot of the snow cover as a function of time, for the full time series, up to the present.

### Commonality of early snowfalls

Last year, in 2020, a heavy snow fell on October 23-24, while there were still many leaves on
the trees.  This caused power outages and brought down many many tree limbs across Moscow (see
evidence of this below, with a truck ready to head for one of Many trips to the city's yard
waste dump that day).

<img src="early_snow.jpg" alt="alt text" width="500"/>

How typical are these early season snow storms in Moscow?  Has there been October snow
accumulation in any of the prior 5 or so years?

### Optional: Seasonal cycle of snow cover in Moscow
Make a plot of a typical year, and plot each year's snow cover record on this single, 12 month axis.
I'm thinking of something along the lines of the National Snow and Ice Data Center's
[Charctic Interactive Sea Ice Graph](https://nsidc.org/arcticseaicenews/charctic-interactive-sea-ice-graph/), 
in which an entire time series is plotted on one year, to facilitate comparison and identify typical
patterns.

Tweak the graph you make of the annual snow cover so that it shows the average snow cover
as a function of day.  

